In [1]:
import os
import geopandas

import json
import topojson
from topojson.utils.ops import select_unique_combs
from topojson.utils.ops import junctions_shared_paths

from IPython.display import SVG, display
from shapely import geometry

import numpy as np
%matplotlib inline

In [2]:
topojson.__version__

'1.0rc1'

In [3]:
# ne_highres = '/Users/mattijnvanhoek/Downloads/ne_10m_admin_1_states_provinces/ne_10m_admin_1_states_provinces.shp'
# os.path.isfile(ne_highres)

In [4]:
data = geopandas.read_file(geopandas.datasets.get_path("naturalearth_lowres"))

In [6]:
%%prun -l 10
# present timing split out in the different subtasks
ex = topojson.extract(data)
jo = topojson.join(ex)

287
 

         399290 function calls (388511 primitive calls) in 0.711 seconds

   Ordered by: internal time
   List reduced from 521 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      287    0.219    0.001    0.420    0.001 ops.py:56(junctions_shared_paths)
     1318    0.045    0.000    0.045    0.000 {method 'argsort' of 'numpy.ndarray' objects}
     4646    0.032    0.000    0.035    0.000 predicates.py:23(__call__)
    533/1    0.024    0.000    0.045    0.045 copy.py:236(_deepcopy_dict)
     1419    0.021    0.000    0.062    0.000 arraysetops.py:268(_unique1d)
     1444    0.019    0.000    0.019    0.000 {method 'reduce' of 'numpy.ufunc' objects}
      928    0.018    0.000    0.045    0.000 coords.py:164(__call__)
      859    0.015    0.000    0.093    0.000 arraysetops.py:434(in1d)
     1144    0.015    0.000    0.024    0.000 function_base.py:4270(insert)
13058/12881    0.010    0.000    0.019    0.000 {built-in method num

In [7]:
# 0.711 seconds using Numpy-variant versus 0.953 seconds with the v1.0rc1 version
# Numpy variant is 30% quicker, but with a result of lower quality
# My feeling says its not worth it.
# moreover, it still cannot process high resolution geometric files.

In [ ]:
#jo['junctions']

In [ ]:
g_jc = geometry.MultiPoint(jo['junctions'])
g_ls = geometry.MultiLineString([obj[~np.isnan(obj)[:,0]].tolist() for obj in jo['linestrings']])
geometry.GeometryCollection([g_jc,g_ls])

In [ ]:
np.isin(ls_xy, jc).sum(axis=2)

In [ ]:
ls_xy = jo['linestrings']
ls_xy_roll = np.roll(ls_xy, 1, axis=1)

In [ ]:
eucl_dist = np.sqrt(
    (ls_xy_roll[:, :, 0] - ls_xy[:, :, 0]) ** 2 + (ls_xy_roll[:, :, 1] - ls_xy[:, :, 1]) ** 2
)
eucl_dist[:,0] = 0
ls_cumsum = eucl_dist.cumsum(axis=0)

In [ ]:
# make numpy array from coordinates of extracted linestrings
arraylist = [np.array(g) for g in ex['linestrings']]
# lengths of linestrings
length_geoms = np.array([len(xy) for xy in arraylist])

In [ ]:
# define empty array with size no. linestrings * max. length linestrings * no. coordinates
coord_arr = np.ones((len(arraylist), np.max(length_geoms), 2))*np.nan 
# populate columns
for i, c in enumerate(arraylist):  
    coord_arr[i, :len(c)] = c

In [ ]:
junctions_list = []
for val in combs:
    idx_geom = val[0]
    shared_geoms = val[1]
    junctions = junctions_shared_paths(coord_arr, idx_geom, shared_geoms, length_geoms)
    junctions_list.extend([junctions])
    
#unique_junctions = np.unique(np.array(junctions_list), axis=0)    

In [ ]:
unique_junctions

In [ ]:
length_geom2 = np.array([len(xy) for xy in junctions_list])
len(junctions_list)
coord_ar2 = np.ones((len(junctions_list), np.max(length_geom2), 2))*np.nan 
# populate columns
for i, c in enumerate(junctions_list):  
    coord_ar2[i, :len(c)] = c

In [ ]:
coord_ar2

In [ ]:
np.unique(coord_ar2, )